<a href="https://colab.research.google.com/github/linbinbin/Langchain_practice/blob/main/colab/excel_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- このノートブックの内容は2024年5月時点の [LangChain 公式ドキュメント](https://python.langchain.com/v0.1/docs/get_started/introduction)に依拠しています。
- 事前に社内規定PDFを概要欄リンクからダウンロードし、動画内の説明に従ってアップロードしておいてください。

## 準備：API キーの設定

In [ ]:
!pip install -q openai
!pip install -q pypdf
!pip install -q langchain
!pip install -q langchain-openai
!pip install -q langchain-chroma
!pip install -q docx2txt
!pip install -q langchain_community
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:

from google.colab import userdata
import openai
import os
# OPENAI_API_KEY を取得
openai.api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai.api_key
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = "excel_RAG"
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "Password"

graph = Neo4jGraph()

# RAG（LLM の知識拡張）
RAG を活用することで、LLM は学習データに含まれない知識を得ることができます。

参照用データとしては、現在 LangChain は以下に対応しています：
- .txt
- .csv
- .html
- .json
- .md
- .pdf
- MS Office ファイル

今回は PDF からの知識抽出を試してみましょう。

In [ ]:
#from langchain_community.document_loaders import PyPDFDirectoryLoader
#from langchain_community.document_loaders import UnstructuredWordDocumentLoader
#from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
#loader = Docx2txtLoader("docx/example.docx")
loader = UnstructuredExcelLoader("excel/example.xlsx", mode="elements")
data = loader.load()
#data[1]


## 文埋め込みと類似文書検索

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
data = filter_complex_metadata(data)
# ベクトルデータベースを作る
db = Chroma.from_documents(data, embeddings_model)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":1})

In [ ]:
query = "3月の売り上げは？"
retriever.invoke(query)

## Retrieval を活用したチャットモデルの制御

### まずはただ聞いてみる

In [ ]:
from langchain_openai import ChatOpenAI
gpt4o = ChatOpenAI(model="gpt-4o", temperature=0.1)
out = gpt4o.invoke("3月の売り上げは？")
print(out.content)

申し訳ありませんが、具体的な売り上げデータについては把握しておりません。売り上げに関する情報を知りたい場合は、該当する企業や店舗の公式発表や財務報告書などを参照することをお勧めします。もし他に質問があれば、お知らせください。


### 準備：プロンプトテンプレートを作る

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="以下を参照して、質問に答えてください。\n\n{context}\n\n質問：{question}"
)
example = {"question":"これは質問です", "context":"これは外部知識です"}

prompt_template.invoke(example)

StringPromptValue(text='以下を参照して、質問に答えてください。\n\nこれは外部知識です\n\n質問：これは質問です')

### 準備：抽出器を作る

In [ ]:
# ベクトルデータベースを作る
db = Chroma.from_documents(data, embeddings_model)

# 抽出器を作る
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 1})
retriever.invoke("3月の売り上げは？")

### 準備：フォーマッターを作る
抽出器の出力を生テキストの形に成形する関数を用意しておく。

In [ ]:
def text_formatter(retriever_output):
  raw_text = retriever_output[0].page_content
  raw_text_wo_newline = raw_text.replace("\n", "")
  return raw_text_wo_newline

### 抽出・補完・生成のチェインを作る

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(model="gpt-4o")

retriever、prompt_template、llmが用意できたので、これらを組み合わせて、情報の流れを定義する。LangChainでは、これから書いていくような「インプットからアウトプットまでの流れ」をチェイン（chain）と呼ぶ。

In [ ]:

chain = ({"question":RunnablePassthrough(), "context":retriever|text_formatter}
         | prompt_template
         | llm
         )

In [ ]:
chain.invoke("3月の売上は？")

AIMessage(content='表に直接「3月の売上」が記載されていませんが、「当月まで累計実績」と「3ヶ月後累計推定」から逆算することができます。\n\n「当月まで累計実績」は61.172083、「3ヶ月後累計推定」は120.832083です。これから、3ヶ月分の売上を計算します：\n\\[ 3ヶ月分の売上 = 120.832083 - 61.172083 = 59.66 \\]\n\n次に、この3ヶ月分の売上を3で割ることで、1ヶ月あたりの売上の平均を求めます：\n\\[ 1ヶ月あたりの売上の平均 = 59.66 / 3 ≈ 19.887 \\]\n\nこれは近似値ですが、3ヶ月分の売上が3月、4月、5月のものと仮定すると、この平均値が3月の売上と推定できます。\n\nしたがって、3月の売上はおおよそ19.887と推定されます。', response_metadata={'token_usage': {'completion_tokens': 234, 'prompt_tokens': 230, 'total_tokens': 464}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-d8e56983-68e1-4cd9-9031-d0271448cff8-0', usage_metadata={'input_tokens': 230, 'output_tokens': 234, 'total_tokens': 464})

In [ ]:
resp=chain.invoke("月３、４、５の当月までの累計実績の変化を教えて")

In [ ]:
resp.content

'月3、4、5の当月までの累計実績の変化について、以下のようにまとめます。\n\n### 売上の累計実績\n- **月3**: 96.014928\n- **月4**: 100.049283\n- **月5**: 154.002083\n\n### 部門利益の累計実績\n- **月3**: 17.231823\n- **月4**: 23.017230\n- **月5**: 25.091665\n\n### 部門利益率の累計実績\n- **月3**: 0.172233\n- **月4**: 0.172233\n- **月5**: 0.162931\n\nこれらのデータから、次のような変化が見られます。\n\n#### 売上の累計実績の変化\n- 月3から月4への変化: 増加（96.014928 → 100.049283）\n- 月4から月5への変化: 大幅な増加（100.049283 → 154.002083）\n\n#### 部門利益の累計実績の変化\n- 月3から月4への変化: 増加（17.231823 → 23.017230）\n- 月4から月5への変化: 増加（23.017230 → 25.091665）\n\n#### 部門利益率の累計実績の変化\n- 月3から月4への変化: 変化なし（0.172233 → 0.172233）\n- 月4から月5への変化: 減少（0.172233 → 0.162931）\n\n以上のように、売上と部門利益は全体的に増加している一方で、部門利益率は月5で減少しています。'

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip freeze > request.txt